In [1]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from langchain_anthropic import ChatAnthropic

load_dotenv()
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
#model = ChatAnthropic(model="claude-3-haiku-20240307", anthropic_api_key=anthropic_api_key)
model = ChatAnthropic(model="claude-3-5-sonnet-20240620", anthropic_api_key=anthropic_api_key, max_tokens=4096)
# Set up LangChain tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "sprint-planning"

In [17]:
from typing import Dict

class SprintPlanningState(TypedDict):
    sprint_planning_note: str
    current_backlog: str
    employee_profiles: Dict[str, str]
    selected_backlog_items: List[Dict]
    team_capacity: Dict[str, int]
    tasks: List[Dict]
    task_assignments: Dict[str, List[str]]
    sprint_gantt_chart: str
    personal_gantt_charts: Dict[str, str]
    updated_global_backlog: str
    personal_reports: Dict[str, str]
    sprint_start_date: str
    sprint_end_date: str
    sprint_duration: int
    working_days: int
    average_velocity: int


In [8]:
BACKLOG_ITEM_SELECTOR_PROMPT = """
You are an AI assistant helping with Sprint Planning.
Your task is to select appropriate backlog items for the upcoming sprint and generate new items based on the sprint planning note.

Sprint Planning Note:
####
{sprint_planning_note}
####

Current Backlog:
####
{current_backlog}
####

Please perform the following tasks:
1. Analyze the sprint planning note and the current backlog.
2. Select appropriate items from the current backlog for the upcoming sprint.
3. Generate new user stories, epics, or tasks based on the discussion in the sprint planning note.
4. Prioritize the selected and newly generated items.
5. Provide a brief justification for each selected or generated item.

Output the results in the following JSON format:
{{
  "selected_items": [
    {{
      "id": "item_id",
      "type": "epic/story/task",
      "title": "Item title",
      "description": "Item description",
      "priority": "high/medium/low",
      "justification": "Reason for selection"
    }}
  ],
  "new_items": [
    {{
      "type": "epic/story/task",
      "title": "New item title",
      "description": "New item description",
      "priority": "high/medium/low",
      "justification": "Reason for generation"
    }}
  ]
}}

Ensure that the selected items align with the team's capacity and the sprint's goals.
"""

In [9]:
CAPACITY_CALCULATOR_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to calculate the team's capacity for the upcoming sprint.

Sprint Details:
- Sprint Duration: {sprint_duration} days
- Working Days: {working_days} days

Team Member Profiles:
####
{employee_profiles}
####

Historical Team Velocity:
- Average velocity: {average_velocity} story points per sprint

Please calculate the team's capacity for this sprint:
1. Consider each team member's availability (accounting for time off, other commitments).
2. Use the historical team velocity as a baseline.
3. Adjust the capacity based on any factors mentioned in the employee profiles or sprint details.

Provide the results in the following JSON format:
{{
  "team_capacity": {{
    "total_story_points": number,
    "total_available_hours": number
  }},
  "individual_capacity": {{
    "employee_name": {{
      "available_days": number,
      "estimated_story_points": number,
      "estimated_hours": number
    }}
  }},
  "explanation": "Brief explanation of the capacity calculation and any adjustments made"
}}
"""

In [12]:
TASK_BREAKDOWN_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to break down the selected backlog items into smaller, manageable tasks and provide effort estimates.

Selected Backlog Items:
####
{selected_backlog_items}
####

Team Capacity:
####
{team_capacity}
####

Please perform the following tasks:
1. Break down each backlog item into smaller, specific tasks.
2. Provide an effort estimate for each task in story points or hours.
3. Ensure the total effort aligns with the team's capacity.

Output the results in the following JSON format:
{{
  "tasks": [
    {{
      "parent_item_id": "id of the parent backlog item",
      "task_id": "unique task id",
      "title": "Task title",
      "description": "Task description",
      "estimate": {{
        "unit": "story_points/hours",
        "value": number
      }},
      "dependencies": ["task_id of dependent tasks, if any"]
    }}
  ],
  "total_effort": {{
    "story_points": number,
    "hours": number
  }}
}}

Ensure that the tasks are specific, measurable, and aligned with the Definition of Done.
"""

In [13]:
TASK_ASSIGNER_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to suggest initial task assignments for team members based on their skills and capacity.

Team Member Profiles:
####
{employee_profiles}
####

Tasks:
####
{tasks}
####

Individual Capacity:
####
{individual_capacity}
####

Please perform the following tasks:
1. Analyze each team member's skills and capacity.
2. Suggest task assignments that best match each team member's abilities and available capacity.
3. Ensure a balanced workload across the team.

Output the results in the following JSON format:
{{
  "assignments": {{
    "employee_name": [
      {{
        "task_id": "assigned task id",
        "rationale": "Brief explanation for this assignment"
      }}
    ]
  }},
  "unassigned_tasks": [
    {{
      "task_id": "unassigned task id",
      "reason": "Reason for not assigning"
    }}
  ]
}}

Provide a brief explanation for any tasks left unassigned or any potential overallocation.
"""

In [14]:
SPRINT_VISUALIZER_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to create Gantt chart representations of the sprint plan.

Sprint Details:
- Start Date: {sprint_start_date}
- End Date: {sprint_end_date}

Tasks and Assignments:
####
{tasks_and_assignments}
####

Please perform the following tasks:
1. Create a Mermaid Gantt chart representation for the entire sprint, showing all tasks, their durations, and dependencies.
2. Create individual Mermaid Gantt charts for each team member, showing their assigned tasks.

Output the results in the following format:
{{
  "sprint_gantt_chart": "Mermaid Gantt chart code for the entire sprint",
  "individual_gantt_charts": {{
    "employee_name": "Mermaid Gantt chart code for the individual"
  }}
}}

Ensure that the Gantt charts clearly visualize the sprint timeline, task dependencies, and workload distribution.
"""

In [15]:
GLOBAL_BACKLOG_UPDATER_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to update the global backlog with the results of the sprint planning session.

Current Global Backlog:
####
{current_backlog}
####

Sprint Planning Results:
####
{sprint_planning_results}
####

Please perform the following tasks:
1. Integrate the newly generated items into the global backlog.
2. Update the status of items selected for the sprint.
3. Update effort estimates and priorities based on the sprint planning discussion.
4. Ensure the backlog remains well-organized and prioritized.

Output the updated global backlog in a structured format that can be easily parsed and modified in future sprints.

Provide a brief summary of the changes made to the global backlog.
"""

In [16]:
PERSONAL_REPORT_GENERATOR_PROMPT = """
You are an AI assistant helping with Sprint Planning. Your task is to generate personalized sprint planning reports for each team member.

Team Member: {employee_name}

Employee Profile:
####
{employee_profile}
####

Assigned Tasks:
####
{assigned_tasks}
####

Sprint Gantt Chart:
####
{sprint_gantt_chart}
####

Please create a personalized sprint planning report for this team member. The report should include:
1. A brief overview of the sprint goals and objectives.
2. A list of tasks assigned to the team member, including descriptions and effort estimates.
3. Any specific challenges or areas of focus for the team member this sprint.
4. The team member's personal Gantt chart.
5. Any relevant dependencies or collaborations with other team members.
6. A motivational message encouraging the team member for the upcoming sprint.

Format the report in Markdown, making it easy to read and understand.
"""

In [18]:
import json

def backlog_item_selector_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = BACKLOG_ITEM_SELECTOR_PROMPT.format(
		sprint_planning_note = state.get("sprint_planning_note"),
		current_backlog = state.get("current_backlog"),
	)
    messages = [
        SystemMessage(content=BACKLOG_ITEM_SELECTOR_PROMPT),
        HumanMessage(content="Select and generate backlog items for the sprint.")
	]
    
    response = model.invoke(messages)
    result = json.loads(response.content)
    state['selected_backlog_items'] = result['selected_items'] + result['new_items']
    return state

In [19]:
def capacity_calculator_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = CAPACITY_CALCULATOR_PROMPT.format(
        sprint_duration = state.get("sprint_duration"),
		working_days = state.get("working_days"),
		employee_profiles = json.dumps(state['employee_profiles']),
		average_velocity = state.get("average_velocity"),
	)
    messages = [
		SystemMessage(content=CAPACITY_CALCULATOR_PROMPT),
		HumanMessage(content="Calculate the team's capacity for this sprint.")
	]
    
    response = model.invoke(messages)
    
    result = json.loads(response.content)
    state['team_capacity'] = result['team_capacity']
    state['individual_capacity'] = result['individual_capacity']
    return state

In [20]:
def task_breakdown_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = TASK_BREAKDOWN_PROMPT.format(
        selected_backlog_items=json.dumps(state['selected_backlog_items']),
        team_capacity=json.dumps(state['team_capacity'])
	)
    messages = [
        SystemMessage(content=TASK_BREAKDOWN_PROMPT),
        HumanMessage(content="Break down the selected backlog items into tasks.")
    ]
    
    response = model.invoke(messages)
    
    result = json.loads(response.content)
    state['tasks'] = result['tasks']
    return state

In [21]:
def task_assigner_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = TASK_ASSIGNER_PROMPT.format(
        employee_profiles=json.dumps(state['employee_profiles']),
        tasks=json.dumps(state['tasks']),
        individual_capacity=json.dumps(state['individual_capacity'])
	)
    messages = [
        SystemMessage(content=TASK_ASSIGNER_PROMPT),
        HumanMessage(content="Suggest task assignments for team members.")
    ]
    
    response = model.invoke(messages)
    
    result = json.loads(response.content)
    state['task_assignments'] = result['assignments']
    return state

In [22]:
def sprint_visualizer_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = SPRINT_VISUALIZER_PROMPT.format(
        sprint_start_date=state['sprint_start_date'],
        sprint_end_date=state['sprint_end_date'],
        tasks_and_assignments=json.dumps({
            'tasks': state['tasks'],
            'assignments': state['task_assignments']
        })
	)
    messages = [
        SystemMessage(content=SPRINT_VISUALIZER_PROMPT),
        HumanMessage(content="Create Gantt chart representations of the sprint plan.")
    ]
    
    response = model.invoke(messages)
    
    result = json.loads(response.content)
    state['sprint_gantt_chart'] = result['sprint_gantt_chart']
    state['personal_gantt_charts'] = result['individual_gantt_charts']
    return state

In [23]:
def global_backlog_updater_node(state: SprintPlanningState) -> SprintPlanningState:
    formatted_prompt = GLOBAL_BACKLOG_UPDATER_PROMPT.format(
        current_backlog=state['current_backlog'],
        sprint_planning_results=json.dumps({
            'selected_items': state['selected_backlog_items'],
            'tasks': state['tasks'],
            'assignments': state['task_assignments']
        })
	)
    messages =[
        SystemMessage(content=GLOBAL_BACKLOG_UPDATER_PROMPT),
        HumanMessage(content="Update the global backlog with sprint planning results.")
    ]
    
    response = model.invoke(messages)
    
    state['updated_global_backlog'] = response.content
    return state

In [24]:
def personal_report_generator_node(state: SprintPlanningState) -> SprintPlanningState:
    state['personal_reports'] = {}
    
    for employee, profile in state['employee_profiles'].items():
        formatted_prompt = PERSONAL_REPORT_GENERATOR_PROMPT.format(
            employee_name=employee,
            employee_profile=profile,
            assigned_tasks=json.dumps(state['task_assignments'].get(employee, [])),
            sprint_gantt_chart=state['personal_gantt_charts'].get(employee, '')
		)
        messages = [
            SystemMessage(content=PERSONAL_REPORT_GENERATOR_PROMPT),
            HumanMessage(content=f"Generate a personal sprint planning report for {employee}.")
        ]
        
        response = model.invoke(messages)
        
        state['personal_reports'][employee] = response.content
    
    return state

In [6]:
def create_sprint_planning_workflow(checkpointer):
    workflow = StateGraph(SprintPlanningState)
    
    # Define nodes
    workflow.add_node("backlog_item_selector", backlog_item_selector_node)
    workflow.add_node("capacity_calculator", capacity_calculator_node)
    workflow.add_node("task_breakdown", task_breakdown_node)
    workflow.add_node("task_assigner", task_assigner_node)
    workflow.add_node("sprint_visualizer", sprint_visualizer_node)
    workflow.add_node("global_backlog_updater", global_backlog_updater_node)
    workflow.add_node("personal_report_generator", personal_report_generator_node)
    
    # Define edges
    workflow.add_edge("backlog_item_selector", "capacity_calculator")
    workflow.add_edge("capacity_calculator", "task_breakdown")
    workflow.add_edge("task_breakdown", "task_assigner")
    workflow.add_edge("task_assigner", "sprint_visualizer")
    workflow.add_edge("sprint_visualizer", "global_backlog_updater")
    workflow.add_edge("global_backlog_updater", "personal_report_generator")
    workflow.add_edge("personal_report_generator", END)
    
    # Set entry point
    workflow.set_entry_point("backlog_item_selector")
    
    return workflow.compile(checkpointer=checkpointer)

In [7]:
with SqliteSaver.from_conn_string(":memory:") as memory:
    graph = create_sprint_planning_workflow(memory)
    print(graph.get_graph().draw_mermaid())

%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	backlog_item_selector(backlog_item_selector)
	capacity_calculator(capacity_calculator)
	task_breakdown(task_breakdown)
	task_assigner(task_assigner)
	sprint_visualizer(sprint_visualizer)
	global_backlog_updater(global_backlog_updater)
	personal_report_generator(personal_report_generator)
	__end__([<p>__end__</p>]):::last
	__start__ --> backlog_item_selector;
	backlog_item_selector --> capacity_calculator;
	capacity_calculator --> task_breakdown;
	global_backlog_updater --> personal_report_generator;
	personal_report_generator --> __end__;
	sprint_visualizer --> global_backlog_updater;
	task_assigner --> sprint_visualizer;
	task_breakdown --> task_assigner;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [ ]:
def run_sprint_planning():
    initial_state = SprintPlanningState(
        sprint_planning_note=load_str_from_txt("sprint_planning_note.txt"),
        current_backlog=load_str_from_txt("current_backlog.txt"),
        employee_profiles=load_json_from_file("employee_profiles.json"),
        sprint_start_date="2023-10-05",
        sprint_end_date="2023-10-18",
        sprint_duration=14,
        working_days=10,
        average_velocity=30,
        selected_backlog_items=[],
        team_capacity={},
        tasks=[],
        task_assignments={},
        sprint_gantt_chart="",
        personal_gantt_charts={},
        updated_global_backlog="",
        personal_reports={}
    )

    with SqliteSaver.from_conn_string(":memory:") as memory:
        thread = {"configurable": {"thread_id": "sprint_planning"}}
        graph = create_sprint_planning_workflow(memory)
        for s in graph.stream(initial_state, thread):
            print(s)
        final_state = s

    # Save outputs
    save_str_to_txt(final_state['updated_global_backlog'], "updated_global_backlog.txt")
    save_str_to_txt(final_state['sprint_gantt_chart'], "sprint_gantt_chart.mmd")
    
    for employee, report in final_state['personal_reports'].items():
        save_str_to_txt(report, f"{employee}_sprint_report.md")
        save_str_to_txt(final_state['personal_gantt_charts'][employee], f"{employee}_gantt_chart.mmd")

    print("Sprint planning completed and outputs saved.")

# Run the sprint planning
run_sprint_planning()